# Loading card data

In [ ]:
import json, jsonschema
import time
import tqdm
import numpy as np
import openai
import asyncio

In [7]:
with open("oracle-cards-20250405210637.json", "r") as f:
    data = json.load(f)
print(f"Loaded {len(data)} cards")

Loaded 34458 cards


In [8]:
def get_card_data(oracle_data):
    if type(oracle_data) != dict:
        print(f"Invalid oracle data: {oracle_data}")
        return None
    if "memorabilia" in oracle_data.get("set_type", ""):
        # skip memorabilia cards
        return None
    if "//" in oracle_data.get("name", ""):
        # todo: handle double-faced cards separately
        return None
    if "mana_cost" not in oracle_data or oracle_data["mana_cost"] is None:
        print(f"Missing mana_cost for {oracle_data['name']}")
        return None
    if "oracle_text" not in oracle_data or oracle_data["oracle_text"] is None:
        print(f"Missing oracle_text for {oracle_data['name']}")
        return None
    
    # check if legal in some format
    # legal_formats = oracle_data.get("legalities", []).items()
    # legal_formats = [f for f, l in legal_formats if l == "legal"]
    # if not legal_formats:
    #     print(f"Not legal in any format for {oracle_data['name']}")
    #     return None
    return {
        "name": oracle_data["name"],
        "mana_cost": oracle_data["mana_cost"],
        "cmc": oracle_data["cmc"],
        "type": oracle_data["type_line"],
        "text": oracle_data["oracle_text"],
        "power": oracle_data.get("power"),
        "toughness": oracle_data.get("toughness"),
        "loyalty": oracle_data.get("loyalty"),
        "colors": oracle_data.get("colors"),
        "keywords": oracle_data.get("keywords"),
    }

In [9]:
all_cards_by_name = {card["name"]: get_card_data(card) for card in data}
all_cards_by_name = {k: v for k, v in all_cards_by_name.items() if v is not None}

# Select a subset of cards to process

In [10]:
# A number of classic magic cards
small_set_sample_cards = [
    "Counterspell",
    "Mana Drain",
    "Daze",
    "Force of Will",
    "Spell Snare",
    "Mana Leak",
    "Remand",
    "Spell Pierce",
    "Negate",
    "Dispel",
    "Logic Knot",
    "Dash Hopes",
    "Dovin's Veto",
    "Cancel",
    "Arcane Denial",
    "Counterbalance",
    "Doomsday",
    "Cryptic Command",
    "Swan Song",
    "Flusterstorm",
    "Pact of Negation",
    "Spell Queller",
    "Stubborn Denial",
    "Mystical Dispute",
    "Disdainful Stroke",
    "Unwind",
    "Counterflux",
    "Narset's Reversal",
    "Siren Stormtamer",
    "Mana Tithe",
    "Sphinx's Revelation",
    "Dismember",
    "Swords to Plowshares",
    "Path to Exile",
    "Terminate",
    "Lightning Bolt",
    "Shock",
]

# sample_cards = [card for card in small_set_sample_cards if card in all_cards_by_name]
sample_cards = list(all_cards_by_name.keys())
print(len(sample_cards), "sample cards")

31095 sample cards


# Format card data into text descriptions

In [11]:
def format_card(card_data):
    """Format a card for input into a vector-embedding model."""
    colors = list(map(lambda ch: {"W": "White", "U": "Blue", "B": "Black", "R": "Red", "G": "Green"}.get(ch, ch), card_data.get("colors", [])))

    # header = "The following is a card from the game Magic: The Gathering.\n"
    header = ""
    prompt = f"""{header}
{card_data["name"]}.
{card_data["type"]}.
Mana cost: {card_data["mana_cost"]}
Converted mana cost: {card_data["cmc"]}
Colors: {", ".join(colors) or "Colorless"}.
Oracle text: {card_data["text"] or "None"}
"""
    if card_data["loyalty"] is not None:
        prompt += f"""Loyalty: {card_data["loyalty"]}
"""

    if card_data["power"] is not None and card_data["toughness"] is not None:
        prompt += f"""Power: {card_data["power"]}
Toughness: {card_data["toughness"]}
"""
    if card_data["keywords"] is not None and len(card_data["keywords"]) > 0:
        prompt += f"""Keywords: {", ".join(card_data["keywords"])}
"""
    return prompt.strip()

# sample_cards = list(all_cards_by_name.keys())
formatted_cards = [format_card(all_cards_by_name[card]) for card in sample_cards]
postprocessed_formatted_cards = formatted_cards
print(all_cards_by_name[sample_cards[-10]])
print(postprocessed_formatted_cards[-10])

{'name': 'Curse of Thirst', 'mana_cost': '{4}{B}', 'cmc': 5.0, 'type': 'Enchantment — Aura Curse', 'text': "Enchant player\nAt the beginning of enchanted player's upkeep, this Aura deals damage to that player equal to the number of Curses attached to them.", 'power': None, 'toughness': None, 'loyalty': None, 'colors': ['B'], 'keywords': ['Enchant']}
Curse of Thirst.
Enchantment — Aura Curse.
Mana cost: {4}{B}
Converted mana cost: 5.0
Colors: Black.
Oracle text: Enchant player
At the beginning of enchanted player's upkeep, this Aura deals damage to that player equal to the number of Curses attached to them.
Keywords: Enchant


In [12]:
print([postprocessed_formatted_cards[i] for i, name in enumerate(all_cards_by_name.keys()) if "Planeswalker" in postprocessed_formatted_cards[i]][0])

Tamiyo, Compleated Sage.
Legendary Planeswalker — Tamiyo.
Mana cost: {2}{G}{G/U/P}{U}
Converted mana cost: 5.0
Colors: Green, Blue.
Oracle text: Compleated ({G/U/P} can be paid with {G}, {U}, or 2 life. If life was paid, this planeswalker enters with two fewer loyalty counters.)
+1: Tap up to one target artifact or creature. It doesn't untap during its controller's next untap step.
−X: Exile target nonland permanent card with mana value X from your graveyard. Create a token that's a copy of that card.
−7: Create Tamiyo's Notebook, a legendary colorless artifact token with "Spells you cast cost {2} less to cast" and "{T}: Draw a card."
Loyalty: 5
Keywords: Compleated


In [13]:
for prompt in formatted_cards[:2]:
    print(prompt)
    print("=" * 80)

Nissa, Worldsoul Speaker.
Legendary Creature — Elf Druid.
Mana cost: {3}{G}
Converted mana cost: 4.0
Colors: Green.
Oracle text: Landfall — Whenever a land you control enters, you get {E}{E} (two energy counters).
You may pay eight {E} rather than pay the mana cost for permanent spells you cast.
Power: 3
Toughness: 3
Keywords: Landfall
Static Orb.
Artifact.
Mana cost: {3}
Converted mana cost: 3.0
Colors: Colorless.
Oracle text: As long as this artifact is untapped, players can't untap more than two permanents during their untap steps.


# [Optional][WIP] Use an LLM to expand the card descriptions

In [ ]:
def prepare_postprocessing_prompts(card_descriptions):
    prompts = []
    for text in card_descriptions:
        prompt = f"""
## ROLE
You are an expert Magic: the Gathering rules analyst.

## TASK
Generate compact, retrieval-oriented annotations for the card below.  
Return ONLY the JSON object described in *Output schema* (inside a ```json block).  
Do **not** repeat the card's rules text or name.

## INPUT
{text}
""" + """
## OUTPUT SCHEMA
```json
{
  "mechanics": ["<up to 7 MTG keywords or shorthand different from the card description, e.g. \"ETB\", \"dies trigger\", \"lifegain\" >"],
  "roles": ["<card roles: ramp, removal, finisher, toolbox, etc.>"],
  "strategies": ["<decks or archetypes it fits: aristocrats, blink, etc.>"],
  "synergies": ["<key tribes, card types, or mechanics it combines with>"],
  "power_band":"<one of: weak, fair, strong, broken>",
}
```
"""
        prompts.append(prompt.strip())
    return prompts

postprocessing_prompts = prepare_postprocessing_prompts(formatted_cards)

## Using Qwen locally with vLLM

In [8]:
from vllm import LLM, SamplingParams

sampling_params = SamplingParams(temperature=0.3, top_p=0.95, max_tokens=512)
llm = LLM(model="Qwen/Qwen2.5-7B-Instruct", quantization="fp8")
outputs = llm.chat(messages=[{"role": "user", "content": prompt} for prompt in postprocessing_prompts], sampling_params=sampling_params)

/home/benchislett/Repos/Hexanomicon/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 04-23 15:51:07 [__init__.py:239] Automatically detected platform cuda.


2025-04-23 15:51:07,749	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 04-23 15:51:11 [config.py:689] This model supports multiple tasks: {'score', 'embed', 'generate', 'classify', 'reward'}. Defaulting to 'generate'.
INFO 04-23 15:51:11 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 04-23 15:51:12 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='Qwen/Qwen2.5-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=fp8, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoi

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.85it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.78it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  2.66it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.63it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.67it/s]



INFO 04-23 15:51:15 [loader.py:458] Loading weights took 1.54 seconds
INFO 04-23 15:51:15 [gpu_model_runner.py:1291] Model loading took 8.1372 GiB and 1.889585 seconds
INFO 04-23 15:51:19 [backends.py:416] Using cache directory: /home/benchislett/.cache/vllm/torch_compile_cache/fdfad17ec6/rank_0_0 for vLLM's torch.compile
INFO 04-23 15:51:19 [backends.py:426] Dynamo bytecode transform time: 3.94 s
INFO 04-23 15:51:21 [backends.py:132] Cache the graph of shape None for later use
INFO 04-23 15:51:32 [backends.py:144] Compiling a graph for general shape takes 12.07 s
INFO 04-23 15:51:43 [monitor.py:33] torch.compile takes 16.00 s in total
INFO 04-23 15:51:44 [kv_cache_utils.py:634] GPU KV cache size: 50,512 tokens
INFO 04-23 15:51:44 [kv_cache_utils.py:637] Maximum concurrency for 32,768 tokens per request: 1.54x
INFO 04-23 15:52:21 [gpu_model_runner.py:1626] Graph capturing finished in 37 secs, took 0.49 GiB
INFO 04-23 15:52:21 [core.py:163] init engine (profile, create kv cache, warmup 

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.56s/it, est. speed input: 4342.67 toks/s, output: 27.37 toks/s]


In [16]:
outputs = llm.chat(messages=[[{"role": "user", "content": prompt}] for prompt in postprocessing_prompts], sampling_params=sampling_params)

Processed prompts: 100%|██████████| 37/37 [00:02<00:00, 13.17it/s, est. speed input: 4260.25 toks/s, output: 1113.38 toks/s]


In [36]:
def parse_output(model_output):
    """Parse the model output into a JSON object."""
    try:
        model_output = model_output.strip("```").strip("json").strip()
        return json.loads(model_output)
    except json.JSONDecodeError as e:
        print(f"Failed to parse JSON: {e}")
        return None

print(formatted_cards[2])
print('\n\n\n')
parse_output(outputs[2].outputs[0].text)

The following is a card from the game Magic: The Gathering.

Name: Daze
Mana cost: {1}{U}
Converted mana cost: 2.0
Type Line: Instant
Oracle text: You may return an Island you control to its owner's hand rather than pay this spell's mana cost.
Counter target spell unless its controller pays {1}.
Power: None
Toughness: None
Loyalty: None






{'mechanics': ['Instant', 'Counter', 'Return'],
 'roles': ['Removal', 'Ramp'],
 'strategies': ['Control', 'Blue Weenie'],
 'synergies': ['Counterspells', 'Islands', 'Blue Spells'],
 'power_band': 'medium',
 'why_pick': 'Flexible removal and mana ramp in a single card.'}

In [45]:
print(postprocessing_prompts[2])

## ROLE
You are an expert Magic: the Gathering rules analyst.

## TASK
Generate compact, retrieval-oriented annotations for the card below.  
Return ONLY the JSON object described in *Output schema* (inside a ```json block).  
Do **not** repeat the card's rules text or name.

## INPUT
The following is a card from the game Magic: The Gathering.

Name: Daze
Mana cost: {1}{U}
Converted mana cost: 2.0
Type Line: Instant
Oracle text: You may return an Island you control to its owner's hand rather than pay this spell's mana cost.
Counter target spell unless its controller pays {1}.
Power: None
Toughness: None
Loyalty: None

## OUTPUT SCHEMA
```json
{
  "mechanics": ["<up to 7 MTG keywords or shorthand, e.g. "ETB", "dies trigger", "lifegain" >"],
  "roles":    ["<card roles: ramp, removal, finisher, toolbox, etc.>"],
  "strategies":["<decks or archetypes it fits: aristocrats, blink, etc.>"],
  "synergies":["<key tribes, card types, or mechanics it combines with>"],
  "power_band":"<one of: lo

# Embed the card descriptions

In [13]:
len(formatted_cards)

31095

In [19]:
embedding_client = openai.Client(base_url="http://localhost:30000/v1", api_key="None")

In [ ]:
all_embeddings = []

batch_size = 100
for i in tqdm.tqdm(range(0, len(formatted_cards), batch_size)):
    batch = formatted_cards[i:i + batch_size]
    response = embedding_client.embeddings.create(
        model="Alibaba-NLP/gte-Qwen2-7B-instruct",
        input=batch,
        user="user"
    )
    all_embeddings.extend(map(lambda s: s.embedding, response.data))

len(all_embeddings), len(all_embeddings[0])

100%|██████████| 311/311 [23:46<00:00,  4.59s/it]


(31095, 3584)

In [40]:
all_embeddings_np = np.array(all_embeddings)
all_embeddings_np.shape, all_embeddings_np.dtype

((31095, 3584), dtype('float64'))

In [41]:
np.save("formatted_cards_embeddings.npy", all_embeddings_np)

In [12]:
all_embeddings = np.load("formatted_cards_embeddings.npy")

In [13]:
import textwrap
import chromadb
import pandas as pd
from IPython.display import Markdown
from chromadb import Documents, EmbeddingFunction, Embeddings

In [14]:
chroma_client = chromadb.Client()
db = chroma_client.create_collection(
    "MTGCardsDatabase",
)

In [17]:
assert len(all_embeddings) == len(formatted_cards)
batch_size = 100
for batch in tqdm.tqdm(range(0, len(formatted_cards), batch_size)):
    batch_documents = formatted_cards[batch:batch + batch_size]
    batch_embeddings = all_embeddings[batch:batch + batch_size]
    batch_ids = sample_cards[batch:batch + batch_size]
    db.add(
        documents=batch_documents,
        embeddings=batch_embeddings,
        ids=batch_ids,
    )
# db.add(
#     documents=formatted_cards,
#     embeddings=all_embeddings,
#     ids=sample_cards,
# )

  0%|          | 0/311 [00:00<?, ?it/s]

100%|██████████| 311/311 [00:25<00:00, 12.12it/s]


In [40]:
test_query = "colours Black, Green elf draw a card"
query_embedding = response = embedding_client.embeddings.create(
    model="Alibaba-NLP/gte-Qwen2-7B-instruct",
    input=test_query,
    user="user"
)

In [41]:
query_embedding = np.array(query_embedding.data[0].embedding)
query_embedding

array([-0.01178741,  0.01060486,  0.01307678, ..., -0.02017212,
       -0.01441193, -0.00382042], shape=(3584,))

In [50]:
res = db.query(
    query_embeddings=[query_embedding],
    n_results=20,
)

In [51]:
res["documents"]

[['The following is a card from the game Magic: The Gathering.\n\nName: Young Necromancer\nMana cost: {4}{B}\nConverted mana cost: 5.0\nType Line: <type> Creature — Human Warlock </type>\nColors: <colors> Black </colors>\nOracle text: <oracle_text> When this creature enters, you may exile two cards from your graveyard. When you do, return target creature card from your graveyard to the battlefield. </oracle_text>\nPower: 2\nToughness: 3',
  'The following is a card from the game Magic: The Gathering.\n\nName: Skemfar Shadowsage\nMana cost: {3}{B}\nConverted mana cost: 4.0\nType Line: <type> Creature — Elf Cleric </type>\nColors: <colors> Black </colors>\nOracle text: <oracle_text> When this creature enters, choose one —\n• Each opponent loses X life, where X is the greatest number of creatures you control that have a creature type in common.\n• You gain X life, where X is the greatest number of creatures you control that have a creature type in common. </oracle_text>\nPower: 2\nToughne

In [52]:
# format the documents with the query for re-ranking
documents_string = ""
for i, doc in enumerate(res["documents"][0]):
    documents_string += f"[{i + 1}]: {doc}\n"

rerank_prompt = f"""
You are a language model responsible for re-ranking search findings in an application for finding Magic: The Gathering cards.
Your purpose is to rank documents based on their relevance to the user's query. Consider the query details, the content of the card descriptions, and the context of the game.
The following is a user query: <query> {test_query} </query>. 

Please output only a JSON object with the schema described below. Do not include any other text or explanations. Ensure that the output is valid JSON, and that all documents are included in the ranking exactly once.

Here is an example of the output format assuming there are 10 input documents and they are already in the correct order:
```json
{"{"}
  "ranking": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
{"}"}
```

Here are the documents:\n\n""" + documents_string

llm_client = openai.Client(base_url="http://localhost:30001/v1", api_key="None")
rerank_response = llm_client.chat.completions.create(
    model="google/gemma-3-27b-it",
    messages=[{"role": "user", "content": rerank_prompt}],
)


In [53]:
print(rerank_response)

ChatCompletion(id='8c579c06394545f48325f1bb4a17e7f6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Okay, I understand. You want me to re-rank Magic: The Gathering card results based on a user\'s query. I will act as the re-ranking engine, focusing on relevance to the query, card content, and game context.\n\nHowever, you\'ve provided a very strange and fragmented set of "documents" and a "query." It looks like a lot of text has been chopped up and concatenated. I\'m going to assume you want me to still demonstrate how I would work, so I\'ll treat each of the sections delimited by "text" and "Colors" as a potential card or document. I will also try to infer the user query based on the overall content.\n\n**Inferred User Query:** "Find me cards that interact with or are related to \'life,\' \'sacrifice,\' \'control,\' \'draw,\' \'the,\' \'a,\' \'and,\' \'or,\' \'the,\' \'to,\' \'with,\' and \'this,\' as these words appear repeatedly

In [54]:
def extract_rerank_response(rerank_response):
    """Extract the rerank response from the model output."""
    try:
        rerank_response = rerank_response.choices[0].message.content
        start_index = rerank_response.index("```json") + len("```json") + 1
        # end index is at the ``` after the JSON
        end_index = start_index + rerank_response[start_index:].index("```")
        data = json.loads(rerank_response[start_index:end_index])
        ranking = data["ranking"]
        if not isinstance(ranking, list):
            raise ValueError("Ranking is not a list")
        # if len(ranking) != len(res["documents"][0]):
        #     raise ValueError("Ranking length does not match number of documents")
        # if not len(set(ranking)) == len(ranking):
        #     raise ValueError("Ranking contains duplicates")
        return [i - 1 for i in ranking]  # convert to 0-indexed
    except ValueError as e:
        print(f"Failed to extract JSON: {e}")
        return None

print(extract_rerank_response(rerank_response))

Failed to extract JSON: substring not found
None


In [55]:
[res["documents"][0][i] for i in extract_rerank_response(rerank_response)]

Failed to extract JSON: substring not found


TypeError: 'NoneType' object is not iterable

# [WIP] ColBERT Embeddings

In [9]:
from ragatouille import RAGPretrainedModel
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/home/benchislett/Repos/Hexanomicon/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/benchislett/Repos/Hexanomicon/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


In [14]:
documents = formatted_cards
document_ids = sample_cards
index_path = RAG.index(
    index_name="mtg_cards",
    collection=documents,
    document_ids=document_ids,
    split_documents=False
)

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Apr 26, 21:56:09] #> Creating directory .ragatouille/colbert/indexes/mtg_cards 


[Apr 26, 21:56:10] [0] 		 #> Encoding 30907 passages..


/home/benchislett/Repos/Hexanomicon/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/home/benchislett/Repos/Hexanomicon/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


[Apr 26, 21:56:33] [0] 		 avg_doclen_est = 70.08810424804688 	 len(local_sample) = 30,907
[Apr 26, 21:56:33] [0] 		 Creating 16,384 partitions.
[Apr 26, 21:56:33] [0] 		 *Estimated* 2,179,389 embeddings.
[Apr 26, 21:56:33] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/mtg_cards/plan.json ..
used 20 iterations (0.904s) to cluster 2116213 items into 16384 clusters
[Apr 26, 21:56:34] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/home/benchislett/Repos/Hexanomicon/.venv/lib/python3.12/site-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


[Apr 26, 21:57:04] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


...

2it [00:25, 12.54s/it]
100%|██████████| 2/2 [00:00<00:00, 608.71it/s]

[Apr 26, 21:58:01] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 26, 21:58:01] #> Building the emb2pid mapping..
[Apr 26, 21:58:01] len(emb2pid) = 2179473



100%|██████████| 16384/16384 [00:00<00:00, 257999.57it/s]

[Apr 26, 21:58:01] #> Saved optimized IVF to .ragatouille/colbert/indexes/mtg_cards/ivf.pid.pt


Done indexing!


In [18]:
query = "golgari elves that draw cards"
RAG.search(query, k=400)

/home/benchislett/Repos/Hexanomicon/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


[{'content': 'The following is a card from the game Magic: The Gathering.\n\nGolgari Grave-Troll.\nCreature — Troll Skeleton.\nMana cost: {4}{G}\nConverted mana cost: 5.0\nColors: Green.\nOracle text: This creature enters with a +1/+1 counter on it for each creature card in your graveyard.\n{1}, Remove a +1/+1 counter from this creature: Regenerate this creature.\nDredge 6 (If you would draw a card, you may mill six cards instead. If you do, return this card from your graveyard to your hand.)\nPower: 0\nToughness: 0\nKeywords: Dredge, Mill',
  'score': 21.0625,
  'rank': 1,
  'document_id': 'Golgari Grave-Troll',
  'passage_id': 12639},
 {'content': 'The following is a card from the game Magic: The Gathering.\n\nGolgari Cluestone.\nArtifact.\nMana cost: {3}\nConverted mana cost: 3.0\nColors: Colorless.\nOracle text: {T}: Add {B} or {G}.\n{B}{G}, {T}, Sacrifice this artifact: Draw a card.',
  'score': 20.578125,
  'rank': 2,
  'document_id': 'Golgari Cluestone',
  'passage_id': 28282},


# Create a synthetic dataset for fine-tuning ColBERTv2

In [ ]:
def prepare_synthetic_querygen_prompt(formatted_card: str) -> str:
    prompt = """You are an expert AI assisting in creating a high-quality, diverse synthetic dataset to train Information Retrieval systems for Magic: The Gathering cards.
Your task is to analyse the card description below and generate a set of rich, high-quality potential queries for which the given card would rank very highly.

The output queries should be diverse, covering various aspects of the card, including its mechanics, roles, strategies, and synergies. The queries should be in natural language and should not be too similar to each other.
They should feature a variety of keywords and phrases that a user might use when searching for cards like this one, including specific jargons or slang used in the Magic: The Gathering community. Avoid naming the card directly or repeating its exact text.
You should submit about 5-10 brief queries. Include at least two queries that are short collections of keywords or phrases, and at least two queries that are full sentences.
Your output should be a JSON object with the same schema as the following example:
<schema>
{
    "hypothetical_queries": ["<query1>", "<query2>", "<query3>", "<query4>", "<query5>", "<query6>"]
}
</schema>""".strip() + f"""
<input>
{formatted_card}
</input>
"""
    return prompt.strip()
    
def postprocess_synthetic_querygen_response(model_output: str) -> list[str]:
    try:
        model_output = model_output.strip().strip("```").strip().strip("json").strip()
        data = json.loads(model_output)
        schema = { "type": "object", "properties": { "hypothetical_queries": { "type": "array", "items": { "type": "string" } } }, "required": ["hypothetical_queries"] }
        jsonschema.validate(data, schema)
        return data["hypothetical_queries"]
    except json.JSONDecodeError as e:
        # print(f"Failed to parse JSON: {e}")
        return None
    except jsonschema.ValidationError as e:
        # print(f"Validation error: {e}")
        return None

synthetic_querygen_prompts = [prepare_synthetic_querygen_prompt(card) for card in formatted_cards]

# TODO: Evaluate the model against the base model on a test subset of the card/query pairs

In [15]:
print(synthetic_querygen_prompts[0])

You are an expert AI assisting in creating a high-quality, diverse synthetic dataset to train Information Retrieval systems for Magic: The Gathering cards.
Your task is to analyse the card description below and generate a set of rich, high-quality potential queries for which the given card would rank very highly.

The output queries should be diverse, covering various aspects of the card, including its mechanics, roles, strategies, and synergies. The queries should be in natural language and should not be too similar to each other.
They should feature a variety of keywords and phrases that a user might use when searching for cards like this one, including specific jargons or slang used in the Magic: The Gathering community. Avoid naming the card directly or repeating its exact text.
You should submit about 5-10 brief queries. Include at least two queries that are short collections of keywords or phrases, and at least two queries that are full sentences.
Your output should be a JSON obj

In [ ]:
large_llm_client = openai.AsyncClient(base_url="https://api.centml.com/openai/v1", api_key="API_KEY_HERE")

In [16]:
initial_sample_prompt_idxs = [i for i in range(len(synthetic_querygen_prompts)) if i % 5 != 4]
initial_sample_prompt_pool = [synthetic_querygen_prompts[i] for i in initial_sample_prompt_idxs]
print(len(initial_sample_prompt_pool))

24876


In [17]:
# Load the previous batch from the output file
queries_data = {}
with open("synthetic_queries.jsonl", "r") as f:
    for line in f:
        data: dict = json.loads(line)
        if not data.get("queries", []) or data.get("id") is None:
            print(data.get("card_name"))
            continue
        queries_data[data["id"]] = data["queries"]
print(f"Loaded {len(queries_data)} queries from file")

Loaded 24876 queries from file


In [18]:
batch_size = 256
all_responses = []

In [24]:
len(all_responses)

3

In [27]:
for i in tqdm.tqdm(range(0, len(initial_sample_prompt_pool), batch_size)):
    batch = initial_sample_prompt_pool[i:i + batch_size]
    batch_idxs = initial_sample_prompt_idxs[i:i + batch_size]
    # Check if the batch is already in the output file

    valid_idxs = [idx for idx in batch_idxs if idx not in queries_data]
    valid_positions = [batch_idxs.index(idx) for idx in valid_idxs]
    if not valid_positions:
        continue
    else:
        batch = [batch[j] for j in valid_positions]
        batch_idxs = [batch_idxs[j] for j in valid_positions]
                
    batch_futures = [large_llm_client.chat.completions.create(
        model="deepseek-ai/DeepSeek-V3-0324",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=512,
        n=1,
        stream=False,
        temperature=0.3,
    ) for prompt in batch]
    batch_responses = await asyncio.gather(*batch_futures)
    decoded_responses = [postprocess_synthetic_querygen_response(response.choices[0].message.content) for response in batch_responses]
    all_responses.extend(batch_responses)
    with open("synthetic_queries.jsonl", "a") as f:
        for j, response in enumerate(decoded_responses):
            global_id = batch_idxs[j]
            query_data = {
                "id": global_id,
                "card_name": sample_cards[global_id],
                "queries": response
            }
            f.write(json.dumps(query_data) + "\n")
            queries_data[global_id] = query_data

100%|██████████| 98/98 [00:00<00:00, 114880.32it/s]


In [71]:
batch_responses = await asyncio.gather(*batch_futures)

In [ ]:
for i in range(50):
    idx = initial_sample_prompt_idxs[i]
    # postprocess_synthetic_querygen_response(batch_responses[i].choices[0].message.content)
    print(f"Card  {i}: {sample_cards[idx]}")
    print(f"Description {i}: {formatted_cards[idx]}")
    print(f"Response {i}: {postprocess_synthetic_querygen_response(batch_responses[i].choices[0].message.content)}")

In [29]:
sum([batch_responses[i].usage.completion_tokens + batch_responses[i].usage.prompt_tokens for i in range(len(batch_responses))]) / len(batch_responses)

480.0

## Fine-tuning

In [42]:
from ragatouille import RAGTrainer
trainer = RAGTrainer(model_name="MTGColBERTv_0_0", pretrained_model_name="colbert-ir/colbertv2.0", language_code="en")

/Users/benchislett/Repos/Hexanomicon/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
train_pairs = []
test_pairs = []
train_pairs_idxs = []
test_pairs_idxs = []
idxs_for_train = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
idxs_for_test = []
for idx, queries_for_card in queries_data.items():
    doc = formatted_cards[idx]
    num_queries = len(queries_for_card)
    for i, query in enumerate(queries_for_card):
        if i in idxs_for_train:
            train_pairs.append((query, doc))
            train_pairs_idxs.append((query, idx))
        elif i in idxs_for_test:
            test_pairs.append((query, doc))
            test_pairs_idxs.append((query, idx))
        else:
            print(f"Invalid index {i} for query {query} for card {idx}")

## Mine Negatives

In [22]:
all_cards_by_name

{'Nissa, Worldsoul Speaker': {'name': 'Nissa, Worldsoul Speaker',
  'mana_cost': '{3}{G}',
  'cmc': 4.0,
  'type': 'Legendary Creature — Elf Druid',
  'text': 'Landfall — Whenever a land you control enters, you get {E}{E} (two energy counters).\nYou may pay eight {E} rather than pay the mana cost for permanent spells you cast.',
  'power': '3',
  'toughness': '3',
  'loyalty': None,
  'colors': ['G'],
  'keywords': ['Landfall']},
 'Static Orb': {'name': 'Static Orb',
  'mana_cost': '{3}',
  'cmc': 3.0,
  'type': 'Artifact',
  'text': "As long as this artifact is untapped, players can't untap more than two permanents during their untap steps.",
  'power': None,
  'toughness': None,
  'loyalty': None,
  'colors': [],
  'keywords': []},
 'Sensory Deprivation': {'name': 'Sensory Deprivation',
  'mana_cost': '{U}',
  'cmc': 1.0,
  'type': 'Enchantment — Aura',
  'text': 'Enchant creature\nEnchanted creature gets -3/-0.',
  'power': None,
  'toughness': None,
  'loyalty': None,
  'colors': [

In [24]:
formatted_cards[0], sample_cards[0]

('Nissa, Worldsoul Speaker.\nLegendary Creature — Elf Druid.\nMana cost: {3}{G}\nConverted mana cost: 4.0\nColors: Green.\nOracle text: Landfall — Whenever a land you control enters, you get {E}{E} (two energy counters).\nYou may pay eight {E} rather than pay the mana cost for permanent spells you cast.\nPower: 3\nToughness: 3\nKeywords: Landfall',
 'Nissa, Worldsoul Speaker')

In [30]:
# Easy negatives: rejection-sample random cards from the database as long as at least one of the key attributes is different
easy_negatives = {}
num_negatives_per_card = 1000
for idx in tqdm.tqdm(range(len(formatted_cards))):
    doc_data = all_cards_by_name[sample_cards[idx]]
    # sample 2 * num_negatives_per_card cards from the database
    sample_idxs = np.random.choice(range(len(sample_cards)), size=int(num_negatives_per_card * 1.5), replace=False)
    sample_card_names = [sample_cards[i] for i in sample_idxs]
    sample_cards_data = [all_cards_by_name[name] for name in sample_card_names]
    # filter out cards that match the current card on all key attributes
    filtered_card_names = [card["name"] for card in sample_cards_data if not (
        set(card["colors"]) == set(doc_data["colors"]) and
        card["type"] == doc_data["type"] and
        card["cmc"] == doc_data["cmc"] and
        set(card["keywords"]) == set(doc_data["keywords"])
    )]
    # sample num_negatives_per_card cards from the filtered cards, with replacement if needed
    if len(filtered_card_names) > num_negatives_per_card:
        filtered_card_names = filtered_card_names[:num_negatives_per_card]
    else:
        filtered_card_names = np.random.choice(filtered_card_names, size=num_negatives_per_card, replace=True)
    easy_negatives[doc_data["name"]] = filtered_card_names

100%|██████████| 31095/31095 [00:59<00:00, 522.32it/s]


In [33]:
# save easy negatives to file
with open("easy_negatives.jsonl", "w") as f:
    for card_name, negatives in easy_negatives.items():
        data = {
            "card_name": card_name,
            "negatives": negatives
        }
        f.write(json.dumps(data) + "\n")

In [19]:
# load the embeddings from file
with open("easy_negatives.jsonl", "r") as f:
    easy_negatives = {}
    for line in f:
        data = json.loads(line)
        easy_negatives[data["card_name"]] = data["negatives"]

In [2]:
# Medium negatives: embed the cards using a dense vector embedding model and find the nearest neighbors in the embedding space
from sklearn.neighbors import NearestNeighbors
import numpy as np
num_neighbors = 100
embeddings = np.load("formatted_cards_embeddings.npy")
nbrs = NearestNeighbors(n_neighbors=num_neighbors, algorithm='auto', n_jobs=-1).fit(embeddings)
# find the nearest neighbors for each card
indices = nbrs.kneighbors(embeddings, return_distance=False)

In [24]:
medium_negatives = {}
for idx, card_name in enumerate(sample_cards):
    # get the indices of the nearest neighbors
    neighbor_indices = indices[idx]
    # get the names of the nearest neighbors
    neighbor_names = [sample_cards[i] for i in neighbor_indices]
    # filter out the current card from the neighbors
    neighbor_names = [name for name in neighbor_names if name != card_name]
    medium_negatives[card_name] = neighbor_names

In [27]:
# save the medium negatives to file
with open("medium_negatives.jsonl", "w") as f:
    for card_name, negatives in medium_negatives.items():
        data = {
            "card_name": card_name,
            "negatives": negatives
        }
        f.write(json.dumps(data) + "\n")

In [29]:
# load the medium negatives from file
with open("medium_negatives.jsonl", "r") as f:
    medium_negatives = {}
    for line in f:
        data = json.loads(line)
        medium_negatives[data["card_name"]] = data["negatives"]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

idxs = queries_data.keys()
num_queries = [len(queries_data[idx]) for idx in idxs]
plt.figure(figsize=(10, 6))
sns.histplot(num_queries)
plt.xlabel("Number of queries per card")
plt.ylabel("Number of cards")
plt.title("Distribution of number of queries per card")
plt.show()


## Training

In [37]:
train_triples = []
for query, doc_idx in tqdm.tqdm(train_pairs_idxs):
    # sample 2 easy negatives and 8 medium negatives
    easy_negatives_sample = np.random.choice(easy_negatives[sample_cards[idx]], size=2, replace=False)
    medium_negatives_sample = np.random.choice(medium_negatives[sample_cards[idx]], size=8, replace=False)
    # add the negatives to the triples
    for negative in [*easy_negatives_sample, *medium_negatives_sample]:
        train_triples.append((query, formatted_cards[doc_idx], formatted_cards[sample_cards.index(negative)]))


100%|██████████| 242695/242695 [04:31<00:00, 894.38it/s]


In [ ]:
len(train_triples), len(train_triples) / 32

In [40]:
# save the triples with easy and medium negatives to file
with open("train_triples_easy_med_10x.jsonl", "w") as f:
    for triple in train_triples:
        data = {
            "query": triple[0],
            "positive": triple[1],
            "negative": triple[2],
        }
        f.write(json.dumps(data) + "\n")

In [ ]:
# load the triples from file
with open("train_triples_easy_med.jsonl", "r") as f:
    train_triples = []
    for line in f:
        data = json.loads(line)
        train_triples.append((data["query"], data["positive"], data["negative"]))

In [43]:
trainer.prepare_training_data(raw_data=train_triples, data_out_path="./train_data_v0_0/", all_documents=formatted_cards, mine_hard_negatives=False)

'./train_data_v0_0/'

In [44]:
trainer.train(batch_size=32,
              nbits=4, # How many bits will the trained model use when compressing indexes
              maxsteps=100_000, # Maximum steps hard stop
              use_ib_negatives=True, # Use in-batch negative to calculate loss
              dim=128, # How many dimensions per embedding. 128 is the default and works well.
              learning_rate=5e-6, # Learning rate, small values ([3e-6,3e-5] work best if the base model is BERT-like, 5e-6 is often the sweet spot)
              doc_maxlen=256, # Maximum document length. Because of how ColBERT works, smaller chunks (128-256) work very well.
              use_relu=False, # Disable ReLU -- doesn't improve performance
              warmup_steps="auto", # Defaults to 10%
             )

#> Starting...
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 4,
    "kmeans_niters": 20,
    "resume": false,
    "pool_factor": 1,
    "clustering_mode": "hierarchical",
    "protected_tokens": 0,
    "similarity": "cosine",
    "bsize": 32,
    "accumsteps": 1,
    "lr": 5e-6,
    "maxsteps": 100000,
    "save_every": 14726,
    "warmup": 14726,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": "MTGColBERTv_0_0",
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 256,
    "mask_punctuation": true,
    "checkpoint": "colbert-ir\/colber

Process Process-1:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/benchislett/Repos/Hexanomicon/.venv/lib/python3.12/site-packages/colbert/infra/launcher.py", line 134, in setup_new_process
    return_val = callee(config, *args)
                 ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/benchislett/Repos/Hexanomicon/.venv/lib/python3.12/site-packages/colbert/training/training.py", line 55, in train
    colbert = torch.nn.parallel.DistributedDataParallel(colbert, device_ids=[config.rank],
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/benchislett/Repos/Hexanomicon/.

KeyboardInterrupt: 